#**Download Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project


In [ ]:
%run PhotoshopDetectionCreateCopyPasteDataset.ipynb import *
%run PhotoshopDetectionCreateWarpDataset.ipynb import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset
/content/drive/MyDrive/FALdetector-master
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project


In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

In [ ]:
copypaste_dataloader = data.DataLoader(copypaste_dataset, batch_size=32, shuffle=True, num_workers=0)
warp_dataloader = data.DataLoader(warp_dataset, batch_size=32, shuffle=True, num_workers=0)

# **MVSS-Net**

**Requirements and utils**

In [ ]:
%cd '/content/drive/MyDrive/MVSS-Net-master'

/content/drive/.shortcut-targets-by-id/1I-AV0b1gHt9JC6fXMypktyD0Cjaxuggc/MVSS-Net-master


In [ ]:
# Install NVIDIA apex
!git clone https://github.com/NVIDIA/apex
!pip install -v --disable-pip-version-check --no-cache-dir \
--global-option="--cpp_ext" --global-option="--cuda_ext" ./

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.8/UNKNOWN
sysconfig: /usr/include/python3.8/UNKNOWN
Value for sch

In [ ]:
import os
import io
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image

from models.mvssnet import get_mvss
from models.resfcn import ResFCN
from common.tools import inference_single
from common.utils import calculate_pixel_f1
from apex import amp


**Load a pre-trained MVSS-Net**

---



In [ ]:
mvssnet_path = './ckpt/mvssnet_casia.pt'
model = get_mvss(backbone='resnet50',
                  pretrained_base=True,
                  nclass=1,
                  sobel=True,
                  constrain=True,
                  n_input=3,
                  )
checkpoint = torch.load(mvssnet_path, map_location='cpu')
model.load_state_dict(checkpoint, strict=True)
model.cuda()

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

load pretrain success
----------use sobel-------------
----------use constrain-------------
load pretrain success


MVSSNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset/Collective'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset/Collective


Below code is to split train and test data

In [ ]:
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

from torch.utils.data.sampler import SubsetRandomSampler

# Creating data indices for training and validation splits:
copypaste_dataset_size = len(copypaste_dataset)
indices = list(range(copypaste_dataset_size))
split = int(np.floor(validation_split * copypaste_dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_copypaste_loader = data.DataLoader(copypaste_dataset, batch_size, 
                                           sampler=train_sampler)
validation_copypaste_loader = data.DataLoader(copypaste_dataset, batch_size,
                                                sampler=valid_sampler)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for epoch in range(3):

    model.train()
    for inputs, labels in tqdm(train_copypaste_loader):
        # forward + backward + optimize
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        _, outputs = model(inputs.float())
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

100%|██████████| 516/516 [02:44<00:00,  3.14it/s]


In [ ]:
%cd /content/drive/MyDrive/Computer_Vision_Final_Project/ckpt
torch.save(model.state_dict(), 'mvss.pt')
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset/Collective'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/ckpt
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset/Collective


In [ ]:
# evaluation

model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in tqdm(validation_copypaste_loader):
        inputs = inputs.to(device)
        labels = labels.to(device).detach().cpu()
        _, seg = model(inputs.float())
        seg = torch.sigmoid(seg).detach().cpu()
        if torch.isnan(seg).any() or torch.isinf(seg).any():
            output_max_score = 0.0
        else:
            output_max_score = torch.max(seg).numpy()
        if torch.isnan(labels).any() or torch.isinf(labels).any():
            label_max_score = 0.0
        else:
            label_max_score = torch.max(labels).numpy()

        print(output_max_score, label_max_score)
        if (output_max_score < 0.5 and label_max_score < 0.5) or (output_max_score >= 0.5 and label_max_score >= 0.5):
          test_correct += 1
        test_total += 1
mvssresult = test_correct/test_total
print(mvssresult)

  1%|          | 1/129 [00:00<00:59,  2.14it/s]

0.14064372 0.0


  2%|▏         | 2/129 [00:01<01:09,  1.83it/s]

0.38691106 0.9998999999999999


  2%|▏         | 3/129 [00:01<00:49,  2.53it/s]

...

0.4713855 0.0


 99%|█████████▉| 128/129 [00:46<00:00,  2.39it/s]

0.15599667 0.9998999999999999


100%|██████████| 129/129 [00:46<00:00,  2.76it/s]

0.13602704 0.9998999999999999
0.8294573643410853


In [ ]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/results'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/results


In [ ]:
f = open("mvss.txt", "w")
f.write("accuracy is " + str(mvssresult))
f.close()

In [ ]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project


# **FAL-Detector**

In [ ]:
!pip3 install torch torchvision
!pip install -r '/content/drive/MyDrive/FALdetector-master/requirements.txt'
%cd /content/drive/MyDrive/FALdetector-master/
!bash weights/download_weights.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/FALdetector-master
--2022-12-19 22:49:37--  https://www.dropbox.com/s/rb8zpvrbxbbutxc/global.pth?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/rb8zpvrbxbbutxc/global.pth [following]
--2022-12-19 22:49:37--  https://www.dropbox.com/s/raw/rb8zpvrbxbbutxc/global.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc57e069adb4d958079b1bcea828.dl.dropboxusercontent.com/cd/0/inline/By_l7R4MCtBpxSdCtD0qPC8XHkm9oEWTb0vB3kEKUwUCjH--N2h87dhDEbVGkaS0e0Y1Tq5g1bcZfOKUSltKxPsurQ0vPkN3BaRyv39rwOuT_fLiujtZEFaOOLEd9bQq0src1-

In [ ]:
!python3 global_classifier.py --input_path data/val/modified/flickr_0503.png --model_path weights/global.pth
!python3 local_detector.py --input_path data/val/modified/flickr_0503.png --model_path weights/local.pth --dest_folder out/

Probibility being modified by Photoshop FAL: 99.66%


In [ ]:
!python3 eval.py --dataroot data --global_pth weights/global.pth --local_pth weights/local.pth --gpu_id 0

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
eval.py:115: RuntimeWarning: Mean of empty slice.
  delta_psnr = psnr_after.mean() - psnr_before.mean()
Accuracy:  nan
PSNR increase:  nan


In [ ]:
!ls

data	  global_classifier.py	networks   requirements.txt
eval.py   LICENSE		out	   utils
examples  local_detector.py	README.md  weights


In [ ]:
%cd ./drive/MyDrive/FALdetector-master

[Errno 2] No such file or directory: './drive/MyDrive/FALdetector-master'
/content/drive/MyDrive/FALdetector-master


In [ ]:
!ls

data	  global_classifier.py	networks   requirements.txt
eval.py   LICENSE		out	   utils
examples  local_detector.py	README.md  weights


In [ ]:
from networks.drn_seg import DRNSub

In [ ]:
import argparse
import os
import sys
import torch
from PIL import Image
import torchvision.transforms as transforms

def load_classifier(model_path, gpu_id):
    if torch.cuda.is_available() and gpu_id != -1:
        device = 'cuda:{}'.format(gpu_id)
    else:
        device = 'cpu'
    model = DRNSub(1)
    state_dict = torch.load(model_path, map_location='cpu')
    model.load_state_dict(state_dict['model'])
    model.to(device)
    model.device = device
    model.eval()
    return model

tf = transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])])
def classify_fake(model, img_path, no_crop=False,
                  model_file='utils/dlib_face_detector/mmod_human_face_detector.dat'):
    # Data preprocessing
    im_w, im_h = Image.open(img_path).size
    if no_crop:
        face = Image.open(img_path).convert('RGB')
    else:
        faces = face_detection(img_path, verbose=False, model_file=model_file)
        if len(faces) == 0:
            print("no face detected by dlib, exiting")
            sys.exit()
        face, box = faces[0]
    face = resize_shorter_side(face, 400)[0]
    face_tens = tf(face).to(model.device)

    # Prediction
    with torch.no_grad():
        prob = model(face_tens.unsqueeze(0))[0].sigmoid().cpu().item()
    return prob

In [ ]:
model = load_classifier("weights/global.pth",0)
print(model)
# prob = classify_fake(model, args.input_path, args.no_crop)
# print("Probibility being modified by Photoshop FAL: {:.2f}%".format(prob*100))

DRNSub(
  (base): Sequential(
    (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(3

In [ ]:
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

from torch.utils.data.sampler import SubsetRandomSampler

# Creating data indices for training and validation splits:
warp_dataset_size = len(warp_dataset)
indices = list(range(warp_dataset_size))
split = int(np.floor(validation_split * warp_dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_warp_loader = data.DataLoader(warp_dataset, batch_size, 
                                           sampler=train_sampler)
validation_warp_loader = data.DataLoader(warp_dataset, batch_size,
                                                sampler=valid_sampler)


In [ ]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for epoch in range(3):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    
    model.train()
    for inputs, labels in tqdm(train_warp_loader):
        # forward + backward + optimize
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        
        targets = labels.to(torch.float32)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

100%|██████████| 516/516 [03:13<00:00,  2.67it/s]


In [ ]:
%cd /content/drive/MyDrive/Computer_Vision_Final_Project/ckpt
torch.save(model.state_dict(), 'faldetector.pt')
%cd /content/drive/MyDrive/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/ckpt
/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/Computer_Vision_Final_Dataset


In [ ]:
# evaluation

model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, targets in tqdm(validation_warp_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        prob = model(inputs.float())[0].sigmoid().cpu().item()
        if (prob < .5 and labels[0]) or (prob >= .5 and not labels[0]):
          test_correct += 1
        test_total += 1
falresult = test_correct/test_total
print(falresult)

100%|██████████| 129/129 [00:49<00:00,  2.60it/s]

0.8837209302325582


In [ ]:
%cd '/content/drive/MyDrive/Computer_Vision_Final_Project/results'

/content/drive/.shortcut-targets-by-id/1qRPzc_kkTeNinCAOMoXj6d_nwu-u_HfH/Computer_Vision_Final_Project/results


In [ ]:
f = open("faldetector.txt", "w")
f.write("accuracy is " + str(falresult))
f.close()